# Reading and Exploring the Data

In [1]:
'''
Author: Sriram Yarlagadda
Date Created: 8/26/2016
Note: Please use Python 2.7
'''
import os
import numpy as np
import pandas as pd
os.chdir('C:\Users\syarlag1\Desktop\Loyola-SAH-Analysis') #Set location of home dir here.

### Reading Data

In [2]:
os.chdir('./data')
dataFiles = os.listdir('.')
varLst = ['KEY','LOS','DIED','AGE','SAH','ASA','PT_INT_SAH','PT_INT_SAH_ASA','SCORE']#Names of the variables we need
fullDataDict = {}
for fileName in dataFiles:
    dataStr = fileName[:2] + 'Data'; print dataStr #print to confirm that the data is created
    fullDataDict[dataStr] = pd.DataFrame()
    fullDataDict[dataStr] = pd.read_stata(fileName, columns=varLst)
    fullDataDict[dataStr]['KEY'] = fullDataDict[dataStr]['KEY'].astype(str)#Setting the KEY variable as an object   
    #fullDataDict[dataStr].index = fullDataDict[dataStr]['KEY']; del fullDataDict[dataStr]['KEY'] #Setting 'KEY' as index

CAData
FLData
IAData
NYData
WAData


In [3]:
for key in fullDataDict.keys(): #creating a new state identifier column
    fullDataDict[key]['State'] = pd.Series([str(key)[:2]]*fullDataDict[key].shape[0])
allStateData = pd.concat(fullDataDict.values()) #combing into a single dataframe
allStateData.index = range(allStateData.shape[0])

### Data Preprocessing

In [4]:
allStateData.describe()

,LOS,DIED,AGE,SAH,ASA,PT_INT_SAH,PT_INT_SAH_ASA,SCORE
count,38307.000000,38304.000000,38231.000000,38310.0,38310.000000,38310.0,38310.000000,38310.000000
mean,12.153001,0.204626,58.659569,1.0,0.033908,1.0,0.033908,5.732522
std,37.072215,0.406335,16.465707,0.0,0.180971,0.0,0.180971,10.280618
min,-6666.000000,-9.000000,0.000000,1.0,0.000000,1.0,0.000000,0.298197
25%,3.000000,0.000000,48.000000,1.0,0.000000,1.0,0.000000,0.932394
50%,9.000000,0.000000,58.000000,1.0,0.000000,1.0,0.000000,0.932394
75%,17.000000,0.000000,71.000000,1.0,0.000000,1.0,0.000000,6.958751
max,362.000000,1.000000,111.000000,1.0,1.000000,1.0,1.000000,106.059479


As we can see if have some abberations. For example, the min of LOS is -6666, which is not possible. Hence we quickly scan for other such outliers and remove the corresponding cases.

In [5]:
np.sum(np.array(allStateData < 0), 0)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

As we can see from the above output, only two features -- LOS and DIED -- have negative values. We look at the cases that correspond to these:

In [6]:
allStateData[allStateData['DIED'] < 0]

,KEY,LOS,DIED,AGE,SAH,ASA,PT_INT_SAH,PT_INT_SAH_ASA,SCORE,State
35184,6.20061320052e+12,1.0,-9.0,46.0,1.0,0.0,1.0,0.0,6.958751,CA


In [7]:
allStateData[allStateData['LOS'] < 0]

,KEY,LOS,DIED,AGE,SAH,ASA,PT_INT_SAH,PT_INT_SAH_ASA,SCORE,State
24598,6.20061082797e+12,-6666.0,0.0,67.0,1.0,0.0,1.0,0.0,5.473947,CA


In [8]:
np.sum(np.array(pd.isnull(allStateData)),0)

array([ 0,  3,  6, 79,  0,  0,  0,  0,  0,  0])

There are **89** missing values in our data.

We remove all cases which have such outliers:

In [9]:
for feature in allStateData.columns: 
    allStateData = allStateData[allStateData[feature] >= 0] #removing all the cases that are less than 0 (only 2 in our case)

In [10]:
os.chdir('./..')
allStateData.to_csv('./allStateData.csv')